In [ ]:
from functools import reduce

In [ ]:
name = 'input'

with open(f'{name}.txt') as f:
    t = f.read()
    
binary = ''
for c in t:
    binary += format(int(c, 16), '#06b')[2:]

In [ ]:
def decode_packet(binary, loc):
    version = int(binary[loc:loc+3], 2)
    loc += 3
    type_id = int(binary[loc:loc+3], 2)
    loc += 3
    
    if type_id == 4:
        # literal value packet
        EOP = False
        literal_value = ''
        while not EOP: 

            num = binary[loc:loc+5]
            if num[0] == '1':
                literal_value += num[1:]
            elif num[0] == '0':
                literal_value += num[1:]
                EOP = True
            loc += 5
        result = int(literal_value, 2)
        versions = []
        results = []
    else:
        # operator packet
        length_type = binary[loc]
        loc += 1
        
        if length_type == '0':
            length = binary[loc:loc+15]
            loc += 15
            
            n_bits = int(length, 2)
            versions = []
            type_ids = []
            results = []
            bits_decoded = 0
            while bits_decoded < n_bits:
                v, tid, res, l = decode_packet(binary, loc)
                versions.append(v)
                type_ids.append(tid)
                results.append(res)
                bits_decoded += l - loc
                loc += l - loc
            
        elif length_type == '1':
            length = binary[loc:loc+11]
            loc += 11
            
            n_packets = int(length, 2)
            versions = []
            type_ids = []
            results = []
            packets_decoded = 0
            
            while packets_decoded < n_packets:
                v, tid, res, l = decode_packet(binary, loc)
                versions.append(v)
                type_ids.append(tid)
                results.append(res)
                packets_decoded += 1
                loc += l - loc
        
        if type_id == 0:
            func = (lambda x,y: x+y)
        elif type_id == 1:
            func = (lambda x,y: x*y)
        elif type_id == 2:
            func = (lambda x,y: min(x, y))
        elif type_id == 3:
            func = (lambda x,y: max(x, y))
        elif type_id == 5:
            func = (lambda x,y: int(x>y))
        elif type_id == 6:
            func = (lambda x,y: int(x<y))
        elif type_id == 7:
            func = (lambda x,y: int(x==y))
    
    if results:
        result = reduce(func, results)
    
    return version + sum(versions), type_id, result, loc

In [ ]:
decoded = decode_packet(binary, 0)

## Part 1

In [ ]:
print(decoded[0])

## Part 2

In [ ]:
print(decoded[2])